# VANI

**Initial notebook configuration**

In [20]:
%load_ext autoreload
# %load_ext snakeviz
# %load_ext jupyterflame

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
sys.path.append(os.path.dirname(os.getcwd()))

/g/g92/izzet/projects/iopp/vani-analysis-tool/vani-env/bin/python
3.7.2 (default, Feb 26 2019, 08:59:10) 
[GCC 4.9.3]
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)


In [22]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Import libraries**

In [23]:
from IPython.display import HTML
from vani.analyzer import Analyzer, ClusterOptions, ClusterType

**Initialize VANI Analyzer**

In [24]:
%autoreload 2

# Initialize analyzer
co = ClusterOptions(cluster_type=ClusterType.Local)
vn = Analyzer(n_workers=8, cluster_options=co, debug=True)

/g/g92/izzet/projects/iopp/vani-analysis-tool/vani-env/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42170 instead
  f"Port {expected} is already in use.\n"


Dashboard link: http://127.0.0.1:42170/status


**Analyze logs**

In [25]:
%%time
%autoreload 2

# Analysis configuration
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# log_dir = "/p/lustre1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
log_dir = "/p/lustre1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/lustre1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/lustre1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/lustre1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# Do the analysis
analysis = vn.analyze_parquet_logs(log_dir, max_depth=3, persist_stats=False, stats_file_prefix="cm1_")


Logs read (0.10514177568256855)
---Stats---
Computing stats...
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'max_size': 16.0, 'mean_bw': 73.8296266306836, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280, 'total_size': 21.17757113277912}
-----------


/usr/WS2/iopp/projects/vani-analysis-tool/vani/common/nodes.py:78: RuntimeWarning: invalid value encountered in double_scalars
  score = (np.sum(filter_labels) + np.sum(all_metric_labels))/(len(filter_labels) + len(all_metric_labels))


---Stats---
Computing stats...
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'max_size': 16.0, 'mean_bw': 73.8296266306836, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280, 'total_size': 21.17757113277912}
-----------
---Stats---
Computing stats...
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280}
-----------
CPU times: user 1min 34s, sys: 18.3 s, total: 1min 52s
Wall time: 2min 12s


In [26]:
%%time
%autoreload 2

# Do the analysis
analysis = vn.analyze_parquet_logs(log_dir, max_depth=3, persist_stats=True, stats_file_prefix="cm1_")

Logs read (0.04463761858642101)
---Stats---
Computing stats...
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'max_size': 16.0, 'mean_bw': 73.8296266306836, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280, 'total_size': 21.17757113277912}
-----------
---Stats---
Stats loaded from file
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'max_size': 16.0, 'mean_bw': 73.8296266306836, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280, 'total_size': 21.17757113277912}
-----------
---Stats---
Computing stats...
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280}
-----------
CPU times: user 1min 33s, sys: 18 s, total: 1min 51s
Wall time: 2min 11s


In [27]:
%%time
%autoreload 2

# Do the analysis
analysis = vn.analyze_parquet_logs(log_dir, max_depth=3, persist_stats=True, stats_file_prefix="cm1_")

Logs read (0.05080033466219902)
---Stats---
Stats loaded from file
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'max_size': 16.0, 'mean_bw': 73.8296266306836, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280, 'total_size': 21.17757113277912}
-----------
---Stats---
Stats loaded from file
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'max_size': 16.0, 'mean_bw': 73.8296266306836, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280, 'total_size': 21.17757113277912}
-----------
---Stats---
Stats loaded from file
{'io_time': 0.07585656642913818, 'max_duration': 2.191842555999756, 'total_files': 1547, 'total_ops': 27463, 'total_ranks': 1280}
-----------
CPU times: user 1min 32s, sys: 18.7 s, total: 1min 50s
Wall time: 2min 10s


In [28]:
%%time
analysis.render_tree()

Analysis
├── Timeline - Read
│   └── [0.000-668.306] Size=10 20.03125GB 94.6% | Time=1 0.00003s/p 0.0% | Ops=1 1282 4.7% | Files=10 2 0.1% | Bandwidth=1 1580.68798GB/s 2141.0% | Parallelism=10 1280 100.0% | Xfer=1 16.00000MB 100.0% | Confidence=48.6%
│       ├── [0.000-334.153] Size=10 20.03125GB 94.6% | Time=1 0.00003s/p 0.0% | Ops=1 1282 4.7% | Files=10 2 0.1% | Bandwidth=1 1580.68798GB/s 2141.0% | Parallelism=10 1280 100.0% | Xfer=1 16.00000MB 100.0% | Confidence=48.6%
│       │   ├── [0.000-167.076] Size=10 20.03125GB 94.6% | Time=1 0.00003s/p 0.0% | Ops=1 1282 4.7% | Files=10 2 0.1% | Bandwidth=1 1580.68798GB/s 2141.0% | Parallelism=10 1280 100.0% | Xfer=1 16.00000MB 100.0% | Confidence=48.6%
│       │   │   ├── [0.000-83.538] Size=10 20.03125GB 94.6% | Time=1 0.00003s/p 0.0% | Ops=1 1282 4.7% | Files=10 2 0.1% | Bandwidth=1 1580.68798GB/s 2141.0% | Parallelism=10 1280 100.0% | Xfer=1 16.00000MB 100.0% | Confidence=48.6%
│       │   │   └── [83.538-167.076] Size= | Time= | Ops= | 

In [29]:
%%time
%autoreload 2
filter_group_node = analysis.children[0]
vn.save_filter_group_node_as_flamegraph(filter_group_node=filter_group_node, output_path="flamegraph-cm1.txt")

CPU times: user 6.5 ms, sys: 2 ms, total: 8.5 ms
Wall time: 7.07 ms


In [30]:
%%time
! chmod +x flamegraph.pl
! flamegraph.pl flamegraph-cm1.txt > flamegraph-cm1.svg

CPU times: user 200 ms, sys: 138 ms, total: 338 ms
Wall time: 639 ms


In [31]:
HTML(filename="flamegraph-cm1.svg")